# 📌 Conversor de Planilha do Planner para o Plano de Ação

### Instruções:
1. Faça upload da planilha do Planner (formato `.xlsx`);
2. O sistema converterá automaticamente com base no modelo;
3. O arquivo será baixado com o nome:
**`PLANO DE AÇÃO - PROJETO BAHIA QUE PRODUZ E ALIMENTA_28-07-2025.xlsx`**
4. Você também pode baixar novamente o último arquivo gerado usando o botão abaixo.


In [ ]:
!pip install -q pandas openpyxl ipywidgets
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from datetime import datetime
import os

backup_dir = '/content/backups'
os.makedirs(backup_dir, exist_ok=True)
log_conversoes = []

def converter_planilha():
    clear_output(wait=True)
    uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
    botao = widgets.Button(description='Converter para Plano de Ação')
    baixar_ultimo = widgets.Button(description='⬇️ Baixar Último Arquivo')
    status = widgets.Output()

    def salvar_backup(df_final, nome):
        caminho = f"{backup_dir}/{nome}"
        df_final.to_excel(caminho, index=False)
        backups = sorted(os.listdir(backup_dir), key=lambda x: os.path.getmtime(os.path.join(backup_dir, x)))
        if len(backups) > 5:
            os.remove(os.path.join(backup_dir, backups[0]))

    def on_button_click(b):
        with status:
            status.clear_output()
            if len(uploader.value) == 0:
                print("⚠️ Nenhum arquivo enviado.")
                return
            for nome_arquivo, file_info in uploader.value.items():
                content = file_info['content']
                df = pd.read_excel(pd.io.common.BytesIO(content))

                df_map = pd.read_excel('/mnt/data/Mapeamento_Colunas.xlsx')
                df_final = pd.DataFrame()
                for _, row in df_map.iterrows():
                    origem = row['Coluna do Planner (origem)']
                    destino = row['Coluna na nova planilha (destino)']
                    regra = str(row['Transformação / Regra especial']).strip().lower()
                    if regra == 'valor fixo':
                        df_final[destino] = [row['Formato esperado']] * len(df)
                    elif regra == 'data (dd/mm/yyyy)':
                        df_final[destino] = pd.to_datetime(df[origem], errors='coerce').dt.strftime('%d/%m/%Y')
                    else:
                        df_final[destino] = df.get(origem, '')

                hoje = datetime.now().strftime('%d-%m-%Y')
                nome_final = f"PLANO DE AÇÃO - PROJETO BAHIA QUE PRODUZ E ALIMENTA_{hoje}.xlsx"
                df_final.to_excel(nome_final, index=False)
                files.download(nome_final)
                salvar_backup(df_final, nome_final)
                log_conversoes.append(f"✔️ {nome_final} gerado em {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

    def baixar_ultimo_arquivo(b):
        hoje = datetime.now().strftime('%d-%m-%Y')
        nome_final = f"PLANO DE AÇÃO - PROJETO BAHIA QUE PRODUZ E ALIMENTA_{hoje}.xlsx"
        if os.path.exists(nome_final):
            files.download(nome_final)

    botao.on_click(on_button_click)
    baixar_ultimo.on_click(baixar_ultimo_arquivo)
    display(uploader, botao, baixar_ultimo, status)

converter_planilha()


In [ ]:
from IPython.display import HTML
if log_conversoes:
    display(HTML('<br>'.join(log_conversoes)))
else:
    print("Nenhuma conversão realizada ainda.")